In [ ]:
import py_entitymatching as em
import pandas as pd

In [ ]:
dfa = pd.read_csv("Table_A.csv")
dfb = pd.read_csv("Table_B.csv")
dfc = pd.read_csv("candidate.csv")
dfp = pd.read_csv("prediction_list.csv")

## Blocking

In [ ]:
def transtime(x):
    if pd.isnull(x["Year"]) == False:
        return int(x["Year"][-2:])
    else: 
        return x["Year"]

In [ ]:
Year = dfa.apply(transtime,axis = 1)

In [ ]:
dfa["Year0"]  = Year
dfa = dfa.drop("Year",axis = 1)

In [ ]:
def transtime1(x):
    if pd.isnull(x["Year"]) == False:
        return x["Year"]%100
    else:
        return x["Year"]

In [ ]:
Year1 = dfb.apply(transtime1,axis = 1)

In [ ]:
dfb["Year1"] = Year1

In [ ]:
dfb = dfb.drop("Year",axis = 1)

In [ ]:
df11 = pd.DataFrame(dfc["A_id"])

In [ ]:
df22 = pd.DataFrame(dfc["B_id"])

In [ ]:
df3 = pd.merge(df11, dfa, how='left', left_on="A_id",right_on = "_id")

In [ ]:
df4 = pd.merge(df22, dfb, how='left', left_on="B_id",right_on = "_id")

In [ ]:
dff = pd.concat([df3,df4],axis=1)

In [ ]:
def dif(x):
    return abs((x["Year1"] - x["Year0"]))

Yeardiff = dff.apply(dif,axis = 1)
dff["dif"] = Yeardiff
dff_final = dff[dff["dif"] <= 2]
## blocking with the difference of release time doesn't 2 years
## dff_final is the reduced dataset that contains all the information of two tables, not the reduced candidate set

In [ ]:
B = dff_final.sample(50)## This is the sample from new blocking, where the final density is exactly 0.2

In [ ]:
H = dff_final.sample(400)## This is the 400 sample, the density in the end is 0.265

## Calculating

In [ ]:
import pandas as pd
from scipy.stats import norm
from numpy import sqrt

delta = .05
Z = norm.ppf(1 - (delta / 2))

def estimate_PR(labeled_pairs, reduced_cands, predicted_matches):
    '''
    labeled_pairs - a pandas dataframe with schema id1,id2,label
                    Note label needs to be Boolean

    reduced_cands - a pandas dataframe with schema id1,id2
    predicted_matches - a pandas dataframe with schema id1,id2
    
    return:
        ( (recall lower bound, recall upper bound), (precision lower bound, precision upper bound) )
    '''

    labeled_pairs.drop_duplicates(inplace=True)
    labeled_pairs.columns = ['id1', 'id2', 'label']
    reduced_cands.columns = ['id1', 'id2']
    reduced_cand_set = set(zip(reduced_cands.id1, reduced_cands.id2))
    predicted_matches = set(zip(predicted_matches.id1, predicted_matches.id2))
    
    # estimate the recall
    # number of positives in the labeled sample
    actual_pos = float(labeled_pairs.label.sum())
    # the maximum number of postives in the candidate set
    max_actual_pos = float(actual_pos + len(reduced_cand_set) - len(labeled_pairs))
    
    # true positives in the labeled sample
    true_pos = float(labeled_pairs.apply(lambda x : (x['id1'], x['id2']) in predicted_matches and x['label'], axis=1).sum())
    #estimated recall
    recall = float(true_pos / actual_pos)

    recall_error = Z * sqrt( ((recall * (1 - recall)) / (actual_pos)) * ((max_actual_pos - actual_pos) / (max_actual_pos - 1)) )


    # estimate Precision
    labeled_set  = set(zip(labeled_pairs.id1, labeled_pairs.id2))
    predicted_pos = float(len(labeled_set & predicted_matches))
    
    predicted_pos_in_reduced_cand_set = float(len(reduced_cand_set & predicted_matches))
    
    alpha =  predicted_pos_in_reduced_cand_set / len(predicted_matches)
    precision = alpha * (true_pos / predicted_pos)
    
    precision_error = alpha * Z * sqrt( ((precision * (1 - precision)) / predicted_pos) * (float((len(predicted_matches) - predicted_pos)) / (len(predicted_matches)  - 1)) )

    return ((recall - recall_error, recall + recall_error),
            (precision - precision_error, precision + precision_error))

In [ ]:
d = {"id1":H["A_id"],"id2":H["B_id"]}
label_pairs = pd.DataFrame(data = d)
label = z["label"].apply(bool)## transfer 0 and 1 to TRUE and FALSE
label_pairs["label"] = label
labeled_pairs = label_pairs
label_pairs = label_pairs.reset_index(drop=True)
print(estimate_PR(labeled_pairs, dfcc, dfp))
## The final consequence is ((0.899774426900109, 0.9870180259300797), (0.9507454786427749, 1.0051845278381064))